配分コードのテスト用notebook

In [1]:
# RLest_20231214ver
# matsunaga/res2023
# 観測に事前情報を使わない，最もシンプルなスタンダードモデル（プリズム制約はあり）
# ver10.1での推定をもとに配分する．このファイルでは混雑内生性は考慮しないバージョンで配分してみる
# けど実際には混雑きいてないのでなしで配分しても変わらないのではー
import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

pathは書き換え必要

In [5]:
####### reading data #######
#df_data = pd.read_csv('/home/matsunaga/res2023/data/20230130_17/20230130_1745_18sec.csv')
#search_folder = "/home/matsunaga/res2023/data/20230130_17/20230130_1745_18sec_end162021_over10under20"
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')
#file_list = list(os.listdir(search_folder))
L = len(df_link)
# 観測データは不要で，代わりにODデータをインプットとして読み込む ######
# df_od = pd.read_csv('')
#N = len(df_od)

In [6]:
####### リンク接続行列 ####### 
def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在
    for i in range(n):
        O = link_data.loc[i, 'o'] 
        D = link_data.loc[i, 'd'] 
        for j in range(n):
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = Imaking(df_link)

for i in range(26, 45, 2): # 階段には滞在リンク入れない．向きは考えないので面倒なことは考えない
    I[i-1, i-1] = 0
    I[i, i] = 0

for i in range(46, 49): # 吸収リンクからは吸収リンクにしか接続しない
    I[i-1, :] = 0 
    I[i-1, i-1] = 1 

###### 改札の吸収リンク ######
ddata = [46, 47, 48]
D = len(ddata)

In [7]:
####### パラメタ&タイムステップ設定 #######
####### ここで推定されたパラメタを入れる ####### 経路長（，混雑），時間割引率
x = np.array([-1.3, 0.8])
# sigma = 15 
T = 11 # inputデータによる．10stepで最終リンクには辿り着く人たち，吸収に行くのに+1timestep要すると考えるとT=11が妥当か．
# 20ステップ以内ならT=20;1=21


##########################
###### 経路選択モデル ######
##########################

###### 経路選択モデリングのための時空間プリズム ######
# 起点Oからの到達可能性指示変数ioの定義 kから出発して時刻（遷移回数）t以内に到達できるリンクaがII[t, k, a]=1になっている
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1): 
    Itlist[ts, :, :] = II 
    II = np.dot(II, I)
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数idの定義
Ittlist = np.zeros((T+1, L, L))
for ts in range(1, T+2): 
    Ittlist[ts-1, :, :] = np.transpose(Itlist[(T+1) - ts, :, :]) 

###### 即時効用行列 ###### # 内生性考えないので即時効用行列は時不変
def Mset(x): 
    cost = np.exp(df_link['length']/10 * x[0]) # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy


In [3]:
###### リンク選択確率計算 ###### ### 目的はProbsを返すこと！！
def newPall(x):
    # Probs = np.zeros((T, L)) ## 時間tでのリンクkの期待人数＝個人ごとの選択確率の総和
    Pall = np.zeros((N, T, L, L)) # 個人ごと，時刻ごとの各リンク間遷移確率行列
    beta = x[-1]

    #result_df = measuring_model2(x)
    #rouped = result_df.groupby('userid')
    #df_list = [group for name, group in grouped]
    #df_list = [data.reset_index(drop=True) for data in df_list]    
    
    #### 以降個人ごとに処理 ####（経路データからOD読み込み→プリズム作成→尤度関数計算）
    for i in range(N): # これはinput dataの長さN
        # dfi = df_list[i]
        # Ti = len(dfi) # 個人の観測限界 ## これもinput dataの長さを入れる？？入れなくてもいいかも→入れなくていいな多分
        # 吸収に入ってる人がだんだん増えてるのが見えるのでOKそう
        #a0 = df_od.loc[0, 'k']
        #abs_link = df_od.loc[Ti-1, 'a']
        #userid = int(df_od.loc[0, 'userid'])

        # OD確定 # これinputから読みこむ！
        ao = df_od.loc[i, 'k'] - 1  
        ad = df_od.loc[i, 'a'] -1 

        # OD→プリズム制約Id
        Idrl = np.zeros((T, L, L))

        for ts in range(T):
            if ts == 0:
                Idrl[ts, ao, :] = I[ao, :] # →全部利用可能なのおかしっくね？？
                continue

            alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, ad, :])[0] 
            for a in alist:
                if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, ad, a]: # always True
                    klist = np.where(I[:, a] == 1)[0]

                    for k in klist:
                        if len(np.where(Idrl[ts - 1, :, k] == 1)[0]) != 0:
                            Idrl[ts, k, a] = 1 

        ##### 即時効用 #####
        M = np.zeros((T, L, L))
        for ts in range(T):
            Mts = Idrl[ts, :, :] * Mset(x) # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### 
        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        ##### 選択確率行列 ##### # 尤度計算は不要
        # Pi = Pall[i, :, :, :] #np.zeros((T, L, L)) # 初期設定
        for t in range(T): # 選択確率は各時刻tごとに決まる
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue # ここがbreakになってたのが癌だった

                    Pall[i, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 

        # ここまでで個人iの時刻別の遷移確率行列Piが出揃ったのでLCP(link choice probability)を計算（個人ごと）
        # LCPは遷移確率ではなく各時刻に個人iがリンクkにいる確率を与えている
        #LCP = np.zeros((T, L))
        #for t in range(T):
        #    if t == 0:
        #        LCP[0, ao] += 1 # 最初t=0はa0に居るので，index aoのところが1になる．それ以外は0のまま
        #    else: 
        #        LCP[t, :] = np.dot(LCP[t-1, :], Pi[t, :, :]) # *は要素ごとの積→行列積は@の方がいい？？？
        #    
            # 個人ごとに加算→期待人数
        #    Probs[t, :] += LCP[t, :]
    
    return Pall # 個人ごとの各時刻ごとのリンク遷移確率の一覧がPallに入った


In [4]:
#####################
###### 配分部分 ###### 
#####################

def assignment(Pall):
    columns = ['od', 't', 'k']
    assign_res = pd.DataFrame(columns = columns)

    res_all = np.zeros((N*T, 3)) # N*Tはdf_odの人数が遷移回数T回だけあるので全体としてはN*T

    for i in range(N): # ODペア毎（近藤さんのはOでまとめている）
        od = int(df_od.loc[i, 'od'])
        ao = df_od.loc[i, 'o'] - 1 
        ad = df_od.loc[i, 'd'] - 1 # ここはインプットをそのまま読み込む．ただしlinkidではなくindexにする．

        res_indivi = np.zeros((T, 3)) ## 個人の経路を入れるやつ

        res_indivi[:, 0] = od           # od(userid)列入れる
        res_indivi[:, 1] = np.arange(T) # timestep列入れる
        
        # 累積確率行列を用意
        Pi = Pall[i, :, :, :]
        for t in range(T):
            for k in range(L):
                if k == 0:
                    Pi[t, :, k] = Pi[t, :, k]
                else:
                    Pi[t, :, k] += Pi[t, :, k-1]

        # 乱数発生させて，累積確率行列に基づいてリンクを入れていく（多分）
        ran_list = [random.random() for _ in range(T)]

        # res_indivi[0, 2]は最初に居た地点なのでaoを直接与える．がlinkidを入れたいのでindexから+1するのを忘れないこと！！！！
        res_indivi[0, 2] = ao + 1 # res_indiviの初期値．2つまり第三列がk（滞在リンク）なので

        for t in range(T-1): ######## t=T-2のときのaをt=T-1（最後）の滞在リンクとして入れるので，range(T-1)とする！
            ran_num = ran_list[t]                   # 乱数を与える
            for a in range(L): 
                if ran_num <= Pi[t, k, a]:          # 乱数を元に遷移先を決める
                    k = a                           # aが次（t+1）のkになる．それを次のres_indiviに入れる
                    res_indivi[t+1, 2] = k
                    break #### ここ必須！！！！
            
        res_all = np.vstack((res_all, res_indivi))
        #### ここまでで個人iの選択した経路がklistに時系列で入った

    # 最後res_allの最初の行を消す
    res_all = np.delete(res_all, 0, axis = 0)

    return res_all # これが最終的な出力

配分にODが必要と思ったが，ちょっとここで詰まってしまっている，，